# First Keras project

Keeping it simple. Probably a little too simple, but let's finish this, learn what we learn and move on to something more educational.

(I'm working with Kaggle's MNIST digist data. It's all preprocessed except for one little thing, which is why this is too easy.)

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

keras.__version__

'2.3.0-tf'

In [3]:
#tf.python.client.device_lib.list_local_devices() #verifies GPU type.
tf.config.experimental.list_physical_devices('GPU')#verifies GPU detected

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
try: ## this means that if I re-run all cells, I don't have to wait for pd.read_csv, which is a little slow.
    dataframe.head()
except NameError:
    dataframe = pd.read_csv('data/train.csv')
dev_df = dataframe.sample(n=3000, random_state=1)
train_df = dataframe.drop(dev_df.index)
assert train_df.shape[1] == 785 #should be 784 + 1

In [6]:
## I have some learning to do with datasets. 
## So the cells below don't get used as of 16/11/20.

def dataframe_to_dataset(dataframe, batch_size=64, label='label'):
    ds = tf.data.Dataset.from_tensor_slices((dataframe.drop(label,axis=1).to_numpy(), dataframe[label]))
    ds.shuffle(buffer_size=len(dataframe))
    ds.batch(batch_size)
    return ds

#dev_ds = dataframe_to_dataset(dev_df)
#train_ds = dataframe_to_dataset(train_df)

In [25]:
from tensorflow.keras import layers

##Build the model
inputs = keras.Input(shape=(784))
x = layers.experimental.preprocessing.Rescaling(1./255)(inputs)
x = layers.Dense(100, activation='relu')(x)
x = layers.Dense(100, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)
model = keras.Model(inputs=inputs,outputs=outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
rescaling_1 (Rescaling)      (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 89,610
Trainable params: 89,610
Non-trainable params: 0
_________________________________________________________________


In [26]:
## Cool. 
## Compile and train

model.compile(optimizer="adam", 
                loss="sparse_categorical_crossentropy",
                metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
            )
(X, y) = (train_df.drop('label',axis=1).to_numpy(), train_df['label'].to_numpy())
(X_val, y_val) = (dev_df.drop('label', axis = 1).to_numpy(), dev_df['label'].to_numpy())
assert X.shape[1] == 784
assert X.shape[0] == y.shape[0]
history = model.fit(X, y, epochs=1, batch_size=64, validation_data=(X_val, y_val))

610/610 [==============================] - 2s 4ms/step - loss: 0.3468 - acc: 0.8996 - val_loss: 0.2098 - val_acc: 0.9390


In [27]:
model.fit(X, y, epochs=25, batch_size=64, validation_data=(X_val, y_val))

Epoch 1/25
610/610 [==============================] - 2s 4ms/step - loss: 0.1425 - acc: 0.9570 - val_loss: 0.1442 - val_acc: 0.9573
Epoch 2/25
610/610 [==============================] - 2s 4ms/step - loss: 0.0985 - acc: 0.9699 - val_loss: 0.1309 - val_acc: 0.9613
Epoch 3/25
610/610 [==============================] - 2s 4ms/step - loss: 0.0743 - acc: 0.9772 - val_loss: 0.1048 - val_acc: 0.9710
Epoch 4/25
610/610 [==============================] - 2s 4ms/step - loss: 0.0573 - acc: 0.9827 - val_loss: 0.1149 - val_acc: 0.9680
Epoch 5/25
610/610 [==============================] - 2s 4ms/step - loss: 0.0447 - acc: 0.9863 - val_loss: 0.1202 - val_acc: 0.9647
Epoch 6/25
610/610 [==============================] - 2s 4ms/step - loss: 0.0371 - acc: 0.9884 - val_loss: 0.1090 - val_acc: 0.9710
Epoch 7/25
610/610 [==============================] - 2s 4ms/step - loss: 0.0308 - acc: 0.9899 - val_loss: 0.1201 - val_acc: 0.9673
Epoch 8/25
610/610 [==============================] - 2s 4ms/step - loss: 0.

In [28]:
## We can now babysit by re-running until there's no consistent improvement with 
## training loss.
model.fit(X, y, epochs=2, batch_size=64, validation_data=(X_val, y_val))

Epoch 1/2
610/610 [==============================] - 3s 4ms/step - loss: 0.0071 - acc: 0.9973 - val_loss: 0.1787 - val_acc: 0.9717
Epoch 2/2
610/610 [==============================] - 2s 4ms/step - loss: 0.0053 - acc: 0.9984 - val_loss: 0.2062 - val_acc: 0.9700


## First ever Keras model complete.

That model was doomed to overfit the data and never got consistently better after ~30 epochs. 

Next, I learn to add L2 regularization to reduce bias.